# Artificial Intelligence Nanodegree

## Convolutional Neural Networks

## Capstone Project: Classification of retinal optical coherence tomography diagnosis

---


## Step 0: Verify TensorFlow is running

In [1]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


b'Hello, TensorFlow!'


---
<a id='step0'></a>
## Step 1: Import Datasets

In [2]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    oct_files = np.array(data['filenames'])
    oct_targets = np_utils.to_categorical(np.array(data['target']), 4)
    return oct_files, oct_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('OCT2017-R2/train')
valid_files, valid_targets = load_dataset('OCT2017-R2/valid')
test_files, test_targets = load_dataset('OCT2017-R2/test')

# load list of oct names
oct_names = [item[20:-1] for item in sorted(glob("OCT2017-R2/train/*/"))]

# print statistics about the dataset
print('There are %d total oct categories.' % len(oct_names))
print('There are %s total oct images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training oct images.' % len(train_files))
print('There are %d validation oct images.' % len(valid_files))
print('There are %d test oct images.'% len(test_files))

Using TensorFlow backend.


There are 4 total oct categories.
There are 2681 total oct images.

There are 1184 training oct images.
There are 938 validation oct images.
There are 559 test oct images.


In [3]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [4]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|███████████████████████████████████████████████████████████████████████████████| 559/559 [00:01<00:00, 306.02it/s]


---
<a id='step0'></a>
## Step N: Create benchmark model

### Import Dog Dataset

In the code cell below, we import a dataset of dog images.  We populate a few variables through the use of the `load_files` function from the scikit-learn library:
- `train_files`, `valid_files`, `test_files` - numpy arrays containing file paths to images
- `train_targets`, `valid_targets`, `test_targets` - numpy arrays containing onehot-encoded classification labels 
- `dog_names` - list of string-valued dog breed names for translating labels

In [5]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential


### TODO: Define your architecture.
model = Sequential([
    
    #Locally connected layer containing fewer weights
    #Break the image up into smaller pieces
    #Use 75 filters to identify the most general patterns
    #Use standard kerner_size of 2
    Conv2D(filters=75, kernel_size=2, padding='same', activation='relu', input_shape=(224,224,3)),
    
    #Reduce dimensionality of convolutional layer,
    #Reduce by taking the maximum value in the filter
    MaxPooling2D(pool_size=2),
    
    #Use 100 filters to identify the more specific patterns
    #Use standard kerner_size of 2
    Conv2D(filters=100, kernel_size=2, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    
    #Use 125 filters to identify the more specific patterns
    #Use standard kerner_size of 2
    Conv2D(filters=125, kernel_size=2, padding='same', activation='relu'),
    
    MaxPooling2D(pool_size=2),
    
    
    
    #Conv2D(filters=125, kernel_size=2, padding='same', activation='relu'),    
    #MaxPooling2D(pool_size=2),
    #Conv2D(filters=125, kernel_size=2, padding='same', activation='relu'),    
    #MaxPooling2D(pool_size=2),
    #Conv2D(filters=125, kernel_size=2, padding='same', activation='relu'),    
    #MaxPooling2D(pool_size=2),    
    #Conv2D(filters=125, kernel_size=2, padding='same', activation='relu'),    
    #MaxPooling2D(pool_size=2),

    
    
    
    
    Dropout(0.3),
    Flatten(),
    # Add a softmax activation layer
    Dense(4, activation='softmax')
])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 75)      975       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 75)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 100)     30100     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 100)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 125)       50125     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 28, 28, 125)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 125)       0         
__________

In [6]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 4

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)





Train on 2624 samples, validate on 938 samples
Epoch 1/4
2624/2624 [==============================] - ETA: 27:45 - loss: 1.3919 - acc: 0.25 - ETA: 14:13 - loss: 3.0511 - acc: 0.22 - ETA: 9:41 - loss: 2.6020 - acc: 0.2167 - ETA: 7:25 - loss: 2.2955 - acc: 0.250 - ETA: 6:03 - loss: 2.1275 - acc: 0.220 - ETA: 5:08 - loss: 2.0084 - acc: 0.216 - ETA: 4:29 - loss: 1.9244 - acc: 0.214 - ETA: 3:59 - loss: 1.8474 - acc: 0.237 - ETA: 3:36 - loss: 1.7986 - acc: 0.250 - ETA: 3:17 - loss: 1.7510 - acc: 0.260 - ETA: 3:02 - loss: 1.7214 - acc: 0.263 - ETA: 2:50 - loss: 1.6976 - acc: 0.250 - ETA: 2:39 - loss: 1.6688 - acc: 0.253 - ETA: 2:29 - loss: 1.6478 - acc: 0.260 - ETA: 2:21 - loss: 1.6319 - acc: 0.253 - ETA: 2:14 - loss: 1.6125 - acc: 0.253 - ETA: 2:07 - loss: 1.5961 - acc: 0.261 - ETA: 2:02 - loss: 1.5933 - acc: 0.255 - ETA: 1:57 - loss: 1.5828 - acc: 0.247 - ETA: 1:52 - loss: 1.5692 - acc: 0.255 - ETA: 1:48 - loss: 1.5614 - acc: 0.252 - ETA: 1:44 - loss: 1.5507 - acc: 0.252 - ETA: 1:40 - loss:

2624/2624 [==============================] - ETA: 50s - loss: 0.9754 - acc: 0.60 - ETA: 49s - loss: 0.8149 - acc: 0.67 - ETA: 48s - loss: 0.7844 - acc: 0.71 - ETA: 49s - loss: 0.7053 - acc: 0.77 - ETA: 48s - loss: 0.7137 - acc: 0.77 - ETA: 48s - loss: 0.7108 - acc: 0.75 - ETA: 47s - loss: 0.7103 - acc: 0.75 - ETA: 47s - loss: 0.7025 - acc: 0.75 - ETA: 47s - loss: 0.7019 - acc: 0.75 - ETA: 46s - loss: 0.6997 - acc: 0.76 - ETA: 46s - loss: 0.7271 - acc: 0.74 - ETA: 45s - loss: 0.7170 - acc: 0.74 - ETA: 45s - loss: 0.7025 - acc: 0.74 - ETA: 45s - loss: 0.7108 - acc: 0.73 - ETA: 44s - loss: 0.6995 - acc: 0.74 - ETA: 44s - loss: 0.7145 - acc: 0.73 - ETA: 43s - loss: 0.7119 - acc: 0.73 - ETA: 43s - loss: 0.6952 - acc: 0.73 - ETA: 43s - loss: 0.6878 - acc: 0.73 - ETA: 42s - loss: 0.6884 - acc: 0.73 - ETA: 42s - loss: 0.6931 - acc: 0.72 - ETA: 41s - loss: 0.6945 - acc: 0.73 - ETA: 41s - loss: 0.7022 - acc: 0.72 - ETA: 41s - loss: 0.6998 - acc: 0.72 - ETA: 40s - loss: 0.6994 - acc: 0.72 - ETA: 

In [8]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

In [9]:
# get index of predicted diagnosis for each image in test set
diagnosis_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(diagnosis_predictions)==np.argmax(test_targets, axis=1))/len(diagnosis_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 57.8000%


---
<a id='step5'></a>
## Step N: Create a CNN to Classify Diagnosis (using Transfer Learning)


- [VGG-19](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogVGG19Data.npz) bottleneck features
- [ResNet-50](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogResnet50Data.npz) bottleneck features
- [Inception](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogInceptionV3Data.npz) bottleneck features
- [Xception](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/DogXceptionData.npz) bottleneck features



In [23]:
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3


ResNet50_model = InceptionV3(weights='imagenet', include_top=False)

In [24]:
from keras.models import Model


# add a global spatial average pooling layer
x = ResNet50_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(4, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(4, activation='softmax')(x)


#x = ResNet50_model.output
#x = Conv2D(filters=75, kernel_size=2, padding='same', activation='relu', input_shape=(224,224,3))(x)
#x = MaxPooling2D(pool_size=2)(x)
#x = Conv2D(filters=100, kernel_size=1, padding='same', activation='relu')(x)
#x = MaxPooling2D(pool_size=2)(x)
#x = Conv2D(filters=125, kernel_size=1, padding='same', activation='relu')(x)    
#x = MaxPooling2D(pool_size=2)(x)
#x = Conv2D(filters=125, kernel_size=1, padding='same', activation='relu')(x)   
#x = MaxPooling2D(pool_size=2)(x)
#x = Conv2D(filters=125, kernel_size=1, padding='same', activation='relu')(x)   
#x = MaxPooling2D(pool_size=2)(x)
#x = Conv2D(filters=125, kernel_size=1, padding='same', activation='relu')(x)    
#x = MaxPooling2D(pool_size=2)(x)    
#x = Conv2D(filters=125, kernel_size=1, padding='same', activation='relu')(x)    
#x = MaxPooling2D(pool_size=2)(x)
#x = Dropout(0.3)(x)
##x = Flatten()(x)
#predictions = Dense(4, activation='softmax')(x)








# this is the model we will train
ResNet50_transfer_model = Model(inputs=ResNet50_model.input, outputs=predictions)

### first: train only the top layers (which were randomly initialized)
### i.e. freeze all convolutional InceptionV3 layers
##for layer in ResNet50_model.layers:
##    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
ResNet50_transfer_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')



In [25]:
from keras.callbacks import ModelCheckpoint  

# train the model on the new data for a few epochs
ResNet50_epochs = 4

ResNet50_checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.ResNet50.hdf5', 
                               verbose=1, save_best_only=True)

ResNet50_transfer_model.fit(train_tensors, train_targets, 
                            validation_data=(valid_tensors, valid_targets),
                            epochs=ResNet50_epochs, batch_size=20, callbacks=[ResNet50_checkpointer], verbose=1)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(ResNet50_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in ResNet50_transfer_model.layers[:174]:
   layer.trainable = False
for layer in ResNet50_transfer_model.layers[174:]:
   layer.trainable = True

from keras.optimizers import SGD
ResNet50_transfer_model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
ResNet50_transfer_model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=ResNet50_epochs, batch_size=20, callbacks=[ResNet50_checkpointer], verbose=1)






Train on 2624 samples, validate on 938 samples
Epoch 1/4


ResourceExhaustedError: OOM when allocating tensor with shape[20,192,52,52] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: conv2d_8/convolution = Conv2D[T=DT_FLOAT, _class=["loc:@training_5/RMSprop/gradients/conv2d_8/convolution_grad/Conv2DBackpropInput"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](activation_53/Relu, conv2d_8/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: training_5/RMSprop/gradients/batch_normalization_57/cond/batchnorm/add_1_grad/Shape/_14165 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_6525_...grad/Shape", tensor_type=DT

In [ ]:
# load the weights that yielded the best validation accuracy# load t 
model.load_weights('saved_models/weights.best.ResNet50.hdf5')

### Load the Model with the Best Validation Loss

In [ ]:
from extract_bottleneck_features import *

from keras.applications.resnet50 import preprocess_input

#return ResNet50(weights='imagenet', include_top=False).predict(preprocess_input(tensor))


def transfer_learning_model_make_prediction(img_path):
    # extract bottleneck features
    bottleneck_feature = preprocess_input(path_to_tensor(img_path))
    # obtain predicted vector
    predicted_vector = model.predict(bottleneck_feature)
    # return dog breed that is predicted by the model
    return np.argmax(predicted_vector)


file_to_test = test_files[800]

print(oct_names)
print(file_to_test)
print(transfer_learning_model_make_prediction(file_to_test))

### Test the Model

In [ ]:


print(test_targets)

# get index of predicted dog breed for each image in test set
transfer_learning_model_predictions = [transfer_learning_model_make_prediction(file) for file in test_files]

# report test accuracy
test_accuracy = 100*np.sum(np.array(transfer_learning_model_predictions)==np.argmax(test_targets, axis=1))/len(transfer_learning_model_predictions)

print('Test accuracy: %.4f%%' % test_accuracy)